In [2]:
import numpy as np
import pandas as pd
%matplotlib inline

In [3]:
def GRR(df, LSL, USL):
    columns = df.columns
    """columns[0]=SN"""
    dfPartMean = df.groupby(columns[0]).mean()

    """columns[1]=Operator"""
    dfMean = df.groupby([columns[1], columns[0]]).mean()

    """columns[2]=Metrics"""
    Xbar = dfMean.groupby(columns[1])[columns[2]].mean()

    dfMax = df.groupby([columns[1], columns[0]]).max()
    dfMin = df.groupby([columns[1], columns[0]]).min()
    Range = dfMax[columns[2]] - dfMin[columns[2]]
    Rbar = Range.groupby(columns[1]).mean()
    Rbarbar = Rbar.mean()
    Xbardiff = Xbar.max() - Xbar.min()
    RP = (dfPartMean.max() - dfPartMean.min())[columns[2]]
    D4 = 2.58
    UCL = D4 * Rbarbar
    K1 = 0.5908
    EV = Rbarbar * K1
    numOfparts = 10
    numOftrials = 3
    K2 = 0.5231
    #     print Xbardiff
    #     print "(Xbardiff*K2)**2-EV**2/(numOfparts*numOftrials) : ",(Xbardiff*K2)**2-EV**2/(numOfparts*numOftrials)
    AV = np.sqrt((Xbardiff * K2) ** 2 - EV ** 2 / (numOfparts * numOftrials))
    if np.isnan(AV):
        AV = 0
    GRR = np.sqrt(EV ** 2 + AV ** 2)
    if np.isnan(GRR):
        GRR = 0
    K3 = 0.3146
    PV = RP * K3
    TV = np.sqrt(GRR ** 2 + PV ** 2)

    #
    # print dfPartMean
    # print dfMean
    # print Xbar
    # print dfMax[columns[2]]-dfMin[columns[2]]
    # print Range
    # print Xbar
    # print Rbar
    # print Rbarbar
    # print Xbardiff
    # print UCL
    # print RP
    print 'EV -->', EV
    print 'AV -->', AV
    print 'GRR -->', GRR
    print 'PV -->', PV
    print 'TV -->', TV
    print 'ndc -->', 1.41 * PV / GRR
    ndc = np.ceil(1.41 * PV / GRR)
    if np.isnan(ndc):
        ndc = 0
    print 'precision/tolerance ratio --> ', round(600 * GRR / (USL - LSL), 3)
    if np.isnan(round(600 * GRR / (USL - LSL), 3)):
        return EV, AV, GRR, PV, TV, ndc, 0
    else:
        return EV, AV, GRR, PV, TV, ndc, round(600 * GRR / (USL - LSL), 3)


In [4]:
filename = r'C:\Users\fanchang\fan_python\G501_COMBO1_N1\G501_COMBO1_N1_phase_10_out.csv'
df = pd.read_csv(filename)

In [5]:
df.head(5)

,Station,sn,Operator,Phase,SFR_110lp_60IH_TL_H,SFR_110lp_60IH_BR_V,SFR_110lp_30IH_BL_V,SFR_110lp_60IH_TR_V,SFR_110lp_80IH_BL_H,SFR_110lp_80IH_BL_V,...,SFR_110lp_80IH_BR_V,SFR_110lp_80IH_BR_H,SFR_110lp_Center_H,SFR_110lp_60IH_BL_V,corner_balance_IH80_H,SFR_110lp_80IH_TL_H,SFR_110lp_30IH_BR_V,SFR_110lp_30IH_BR_H,SFR_110lp_80IH_TL_V,corner_balance_IH80_V
0,G501_COMBO1_N1,91MBA01772,OP1,FrontSfrTestInfCollect,0.5879,0.6261,0.6872,0.6467,0.4604,0.5650,...,0.5386,0.4927,0.7091,0.6317,0.1135,0.4807,0.6835,0.6622,0.5435,0.0467
1,G501_COMBO1_N1,91MBA01772,OP1,FrontSfrTestInfCollect,0.5955,0.6273,0.6950,0.6453,0.4593,0.5655,...,0.5460,0.4997,0.7023,0.6307,0.1189,0.4735,0.6890,0.6554,0.5437,0.0385
2,G501_COMBO1_N1,91MBA01772,OP1,FrontSfrTestInfCollect,0.6001,0.6234,0.6929,0.6409,0.4644,0.5595,...,0.5517,0.4925,0.7096,0.6236,0.1040,0.4725,0.6846,0.6657,0.5459,0.0243
3,G501_COMBO1_N1,91MBA01772,OP2,FrontSfrTestInfCollect,0.5913,0.6194,0.6959,0.6375,0.4576,0.5677,...,0.5426,0.4902,0.7059,0.6237,0.1071,0.4557,0.6847,0.6660,0.5412,0.0467
4,G501_COMBO1_N1,91MBA01772,OP2,FrontSfrTestInfCollect,0.5936,0.6169,0.6927,0.6386,0.4506,0.5663,...,0.5381,0.4884,0.7058,0.6357,0.1284,0.4483,0.6790,0.6608,0.5406,0.0505


In [6]:
columns = df.columns
columns

Index([u'Station', u'sn', u'Operator', u'Phase', u'SFR_110lp_60IH_TL_H',
       u'SFR_110lp_60IH_BR_V', u'SFR_110lp_30IH_BL_V', u'SFR_110lp_60IH_TR_V',
       u'SFR_110lp_80IH_BL_H', u'SFR_110lp_80IH_BL_V', u'SFR_110lp_30IH_BL_H',
       u'SFR_110lp_60IH_TR_H', u'SFR_110lp_60IH_BR_H', u'SFR_110lp_60IH_TL_V',
       u'SFR_110lp_30IH_TR_H', u'SFR_110lp_30IH_TL_V', u'SFR_110lp_80IH_TR_V',
       u'SFR_110lp_80IH_TR_H', u'SFR_110lp_30IH_TL_H', u'SFR_110lp_30IH_TR_V',
       u'SFR_110lp_60IH_BL_H', u'SFR_110lp_Center_V', u'SFR_110lp_80IH_BR_V',
       u'SFR_110lp_80IH_BR_H', u'SFR_110lp_Center_H', u'SFR_110lp_60IH_BL_V',
       u'corner_balance_IH80_H', u'SFR_110lp_80IH_TL_H',
       u'SFR_110lp_30IH_BR_V', u'SFR_110lp_30IH_BR_H', u'SFR_110lp_80IH_TL_V',
       u'corner_balance_IH80_V'],
      dtype='object')

In [7]:
len(columns)

32

In [8]:
"""columns[1]=SN"""
dfPartMean = df.groupby(columns[0]).mean()
dfPartMean

,SFR_110lp_60IH_TL_H,SFR_110lp_60IH_BR_V,SFR_110lp_30IH_BL_V,SFR_110lp_60IH_TR_V,SFR_110lp_80IH_BL_H,SFR_110lp_80IH_BL_V,SFR_110lp_30IH_BL_H,SFR_110lp_60IH_TR_H,SFR_110lp_60IH_BR_H,SFR_110lp_60IH_TL_V,...,SFR_110lp_80IH_BR_V,SFR_110lp_80IH_BR_H,SFR_110lp_Center_H,SFR_110lp_60IH_BL_V,corner_balance_IH80_H,SFR_110lp_80IH_TL_H,SFR_110lp_30IH_BR_V,SFR_110lp_30IH_BR_H,SFR_110lp_80IH_TL_V,corner_balance_IH80_V
Station,,,,,,,,,,,,,,,,,,,,,
G501_COMBO1_N1,0.550504,0.562359,0.678086,0.60301,0.45714,0.560092,0.651864,0.555724,0.575752,0.573183,...,0.486437,0.459754,0.689406,0.606817,0.180496,0.461778,0.656514,0.639723,0.510319,0.207896


In [11]:
"""columns[2]=Operator"""
dfMean = df.groupby([columns[1+1], columns[0+1]]).mean()
dfMean.head()

SFR_110lp_60IH_TL_H  SFR_110lp_60IH_BR_V  \
Operator sn                                                     
OP1      91MBA01772             0.594500             0.625600   
         91MBA01803             0.568167             0.548467   
         91MBA01805             0.576000             0.587133   
         91MBA01816             0.542567             0.499600   
         91MBA01824             0.598733             0.619200   

                     SFR_110lp_30IH_BL_V  SFR_110lp_60IH_TR_V  \
Operator sn                                                     
OP1      91MBA01772             0.691700             0.644300   
         91MBA01803             0.696833             0.590300   
         91MBA01805             0.679067             0.619433   
         91MBA01816             0.634300             0.468833   
         91MBA01824             0.688467             0.660033   

                     SFR_110lp_80IH_BL_H  SFR_110lp_80IH_BL_V  \
Operator sn                                                     
OP1      91MBA01772             0.461367             0.563333   
         91MBA01803             0.483767             0.587200   
         91MBA01805             0.475500             0.566067   
         91MBA01816             0.387500             0.543867   
         91MBA01824             0.447467             0.523067   

                     SFR_110lp_30IH_BL_H  SFR_110lp_60IH_TR_H  \
Operator sn                                                     
OP1      91MBA01772             0.680900             0.611033   
         91MBA01803             0.677767             0.548033   
         91MBA01805             0.676100             0.569367   
         91MBA01816             0.556067             0.494433   
         91MBA01824             0.651867             0.607200   

                     SFR_110lp_60IH_BR_H  SFR_110lp_60IH_TL_V  \
Operator sn                                                     
OP1      91MBA01772             0.624167             0.597967   
         91MBA01803             0.581733             0.600933   
         91MBA01805             0.571967             0.587300   
         91MBA01816             0.526000             0.543900   
         91MBA01824             0.615200             0.602400   

                             ...            SFR_110lp_80IH_BR_V  \
Operator sn                  ...                                  
OP1      91MBA01772          ...                       0.545433   
         91MBA01803          ...                       0.478833   
         91MBA01805          ...                       0.525033   
         91MBA01816          ...                       0.450233   
         91MBA01824          ...                       0.557333   

                     SFR_110lp_80IH_BR_H  SFR_110lp_Center_H  \
Operator sn                                                    
OP1      91MBA01772             0.494967            0.707000   
         91MBA01803             0.497000            0.700900   
         91MBA01805             0.454067            0.725333   
         91MBA01816             0.388667            0.647433   
         91MBA01824             0.497267            0.692733   

                     SFR_110lp_60IH_BL_V  corner_balance_IH80_H  \
Operator sn                                                       
OP1      91MBA01772             0.628667               0.112133   
         91MBA01803             0.610067               0.138333   
         91MBA01805             0.615767               0.084433   
         91MBA01816             0.604000               0.133600   
         91MBA01824             0.616100               0.141467   

                     SFR_110lp_80IH_TL_H  SFR_110lp_30IH_BR_V  \
Operator sn                                                     
OP1      91MBA01772             0.475567             0.685700   
         91MBA01803             0.481700             0.662000   
         91MBA01805             0.496000             0.657533   
         91MBA01816        

In [13]:
"""columns[2]=Metrics"""
Xbar = dfMean.groupby(columns[1+1])[columns[2+2]].mean()
Xbar.head()

Operator
OP1    0.554187
OP2    0.549713
OP3    0.547613
Name: SFR_110lp_60IH_TL_H, dtype: float64

In [ ]:
plt